# Descargar automáticamente del SMN
A partir de la ejecución del siguiente código: 
1. Se descargan los archivos de radiación del Servicio Meteorológico Nacional de las fechas seleccionadas.
2. Se estandarizan los mismos y compila en una única tabla.
3. Se los exporta como archivo CSV.

Fuente: https://www.smn.gob.ar/descarga-de-datos

### R        Datos de Radiación de Buenos Aires y de Ushuaia
Datos que se obtienen mediante la medición de la radiación solar y su interacción con la atmósfera.

In [1]:
#Cargar librerías
import requests
import pandas as pd
from datetime import datetime, date, timedelta
from os import  mkdir, rmdir, remove, path

In [2]:
# Definir fechas
sdate = date(2018, 2, 1)   # fecha de inicio (año, mes, día)
edate = date(2018, 2, 28)   # fecha de fin (año, mes, día)

delta = edate - sdate

In [3]:
# Descarga de datos
dir = "radiacion" #carpeta de destino de los archivos
mkdir(str(dir)) 

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    url = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=radiacionsolar/radsolar' + date + '.txt'
    r = requests.get(url, allow_redirects=True)
    if 'El archivo no existe.' in str(r.content):
        print(date + " does not exist")
    else:
        with open('./' + dir + '/' + date + '.txt', 'wb') as f: f.write(r.content)

20180209 does not exist
20180210 does not exist
20180211 does not exist
20180212 does not exist
20180213 does not exist
20180228 does not exist


In [4]:
#Combinación de archivos
out = pd.DataFrame(columns = ['Horario', 'Global_BsAs[W/m2]', 'Difusa_BsAs[W/m2]', 'Global_Ush[W/m2]', 'Difusa_Ush[W/m2]'])

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    if path.isfile('./' + dir + '/'  + date + '.txt'):
        df1 = pd.read_csv('./' + dir + '/' + date + '.txt', header = None, index_col = False, skiprows= 1,
                    names = ['Horario', 'Global_BsAs[W/m2]', 'Difusa_BsAs[W/m2]', 'Global_Ush[W/m2]', 'Difusa_Ush[W/m2]'])
        out = out.append(df1, ignore_index=True)
    else:
        print(date + " does not exist")
                           
    
display(out.head())
display(out.tail())

20180209 does not exist
20180210 does not exist
20180211 does not exist
20180212 does not exist
20180213 does not exist
20180228 does not exist


,Horario,Global_BsAs[W/m2],Difusa_BsAs[W/m2],Global_Ush[W/m2],Difusa_Ush[W/m2]
0,2018-02-01 00:01:00,-2.758,-3.38,8.835,-5.732
1,2018-02-01 00:02:00,-2.758,-3.38,9.237,-4.204
2,2018-02-01 00:03:00,-2.758,-3.38,9.639,-3.057
3,2018-02-01 00:04:00,-2.758,-3.38,9.237,-2.420
4,2018-02-01 00:05:00,-2.758,-3.38,8.233,-3.439


,Horario,Global_BsAs[W/m2],Difusa_BsAs[W/m2],Global_Ush[W/m2],Difusa_Ush[W/m2]
31631,2018-02-27 23:54:00,-2.145,-1.988,-0.402,-14.013
31632,2018-02-27 23:55:00,-2.349,-1.988,-0.602,-13.631
31633,2018-02-27 23:56:00,-2.349,-1.988,-0.602,-13.376
31634,2018-02-27 23:57:00,-2.349,-1.988,-0.602,-13.758
31635,2018-02-27 23:58:00,-2.349,-1.988,-0.602,-12.994


In [5]:
#Estandarización
df = out

df['Horario'] = pd.to_datetime(df['Horario'], format ='%Y-%m-%d %H:%M:%S') 

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31636 entries, 0 to 31635
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Horario            31636 non-null  datetime64[ns]
 1   Global_BsAs[W/m2]  31636 non-null  float64       
 2   Difusa_BsAs[W/m2]  31636 non-null  float64       
 3   Global_Ush[W/m2]   30198 non-null  float64       
 4   Difusa_Ush[W/m2]   30198 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 1.2 MB
None


In [6]:
#Borra archivos anteriores y exporta la tabla
file = 'datosRadiacion.csv' #nombre del archivo de destino

df.to_csv(file, index = False) 

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    if path.isfile('./' + dir + '/'  + date + '.txt'):
        remove('./' + dir + '/' + date + '.txt')
    else:
        print(date + " does not exist")
        
rmdir(str(dir))

20180209 does not exist
20180210 does not exist
20180211 does not exist
20180212 does not exist
20180213 does not exist
20180228 does not exist
